In [2]:
!pip install razdel

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [5]:
news = open('lenta.txt', encoding = 'utf-8').read()

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [40]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [41]:
sentences_news = sentences_news[24:]

In [42]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, 2))
    trigrams_news.update(ngrammer(sentence, 3))

In [43]:
# матрица
matrix_news = lil_matrix((len(bigrams_news),
                         len(unigrams_news)))

# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word = list(unigrams_news)
word2id = {word:i for i, word in enumerate(id2word)}

id2bigram = list(bigrams_news)
bigram2id = {bigram:i for i, bigram in enumerate(id2bigram)}

In [44]:
bigrams_news['<start> <start>']

32226

In [45]:
# заполняем матрицу
for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id[bigram], word2id[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

In [46]:
def generate(matrix, id2word, bigram2id, n=50, start='<start> <start>'):
    text = []
    current_bigram = start

    for i in range(n):
        current_bigram_idx = bigram2id[current_bigram]

        chosen_word_idx = np.random.choice(matrix.shape[1], p=matrix[current_bigram_idx].toarray()[0])
        chosen_word = id2word[chosen_word_idx]
        text.append(chosen_word)

        current_bigram = current_bigram.split()[1] + ' ' + chosen_word

        if chosen_word == '<end>':
            current_bigram = '<start> <start>'
    return ' '.join(text)

In [47]:
print(generate(matrix_news, id2word, bigram2id).replace('<end>', '\n'))

по словам заместителя генерального прокурора василию колмогорову который таким образом более 20 подароков 
 цена фьючерсов на сырую нефть может привести как к общейсети так и на центральных площадях некоторых городов сел и станиц чечни 
 по неофициальным данным сообщает рбк 
 в норвежском посольстве сказали что дети не будут


In [50]:
def perp(logp, N):
    return np.exp((-1/N) * logp)


def compute_join_trigram(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>', '<start>'] + tokens + ['<end>'], 3):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [51]:
ps_trigram = []
for sent in sent_tokenize(news[:2832]):
    prob, N = compute_join_trigram(sent, bigrams_news, trigrams_news)
    if not N:
        continue
    ps_trigram.append(perp(prob, N))

np.mean(ps_trigram)

96998.69162423717

In [52]:
# новость
phrase = 'По словам журналистов, в корпорации публично не раскрывают причину, однако представители Tesla сообщают ее в личном разговоре с покупателями автомобиля. '
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perp(log_prob, N)

35440.455250719046

In [56]:
# твит
phrase = 'итак из нового интервью моргенштерна мы узнали что дилара которая была с ним ещё до известности всегда хотела денег а онлифанщица которой он оплачивает курсы в лондоне арендует дом и покупает подарки не из-за этого'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perp(log_prob, N)

67262.94860504026

In [57]:
# твит
phrase = 'Кошки уходят на охоту на много часов, и думают, что вы тоже охотитесь. А вот то, что вы через 8 часов приходите без единой мыши, сильно подрывает ваш авторитет.'
log_prob, N = compute_join_trigram(phrase, bigrams_news, trigrams_news)
perp(log_prob, N)

72611.17302015454